#  Машинный перевод с использованием рекуррентных нейронных сетей (RNN)

In [1]:
import json
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torch.optim as optim
import torchtext.transforms as T
from torchtext.data.metrics import bleu_score

## Считывание файлов

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/data/RuBQ_2.0_train.json') as f:
    f_train = json.load(f)
with open('/content/drive/MyDrive/data/RuBQ_2.0_test.json') as f:
    f_test = json.load(f)

In [ ]:
X_train, y_train = pd.DataFrame(f_train)['question_text'].tolist(), pd.DataFrame(f_train)['question_eng'].tolist()
X_test, y_test = pd.DataFrame(f_test)['question_text'].tolist(), pd.DataFrame(f_test)['question_eng'].tolist()

In [ ]:
len(X_train), len(X_test)

(2330, 580)

## Создание двух словарей на основе загруженных данных, добавление специальных токенов `<PAD>`, `<SOS>`, `<EOS>`

Для создания словарей используется токенизация с помощью регулярных выражений.\
Так как словари создаются на основе обучающего множества, будет добавлен специальный токен `<UNK>`, чтобы слова, которых нет в словарях, на тестовом множестве заменялись на него.

In [2]:
tokenizer = RegexpTokenizer("\w+")

In [ ]:
ru_vocab = build_vocab_from_iterator(list(map(lambda x: tokenizer.tokenize(x.lower()), X_train)), specials=['<PAD>', '<UNK>', '<SOS>', '<EOS>'])
ru_vocab.set_default_index(1) # <UNK>
len(ru_vocab)

5974

In [ ]:
en_vocab = build_vocab_from_iterator(list(map(lambda x: tokenizer.tokenize(x.lower()), y_train)), specials=['<PAD>', '<UNK>', '<SOS>', '<EOS>'])
en_vocab.set_default_index(1) # <UNK>
len(en_vocab)

4264

**Максимальное количество слов в предложениях на русском языке**

In [ ]:
pd.Series(list(map(lambda x: tokenizer.tokenize(x.lower()), X_train))).str.len().max()

25

**Максимальное количество слов в предложениях на английском языке**

In [ ]:
pd.Series(list(map(lambda x: tokenizer.tokenize(x.lower()), y_train))).str.len().max()

31

## Создание датасетов

Реализуется `__getitem__` таким образом, чтобы он возвращал кортеж `(x, y)`, где x - это набор индексов токенов для предложений на русском языке, а `y` - набор индексов токенов для предложений на английском языке. Используя преобразования, сделаем длины наборов индексов одинаковой фиксированной длины, добавим в начало каждого набора индекс `<SOS>`, а в конец - индекс токена `<EOS>`.

In [ ]:
class RuEnDataset(Dataset):
    def __init__(self, ru_words, en_words, transform1, transform2):
        self.ru_words = ru_words
        self.en_words = en_words
        self.transform1 = transform1
        self.transform2 = transform2
    def __len__(self):
        return len(self.ru_words)

    def __getitem__(self, index):
        X = self.ru_words[index]
        Y = self.en_words[index]
        X_tr = transform1(tokenizer.tokenize(X.lower()))
        Y_tr = transform2(tokenizer.tokenize(Y.lower()))
        return X_tr, Y_tr

In [ ]:
transform1 = T.Sequential(
    T.VocabTransform(ru_vocab),
    T.Truncate(max_seq_len=8),
    T.AddToken(2, begin=True),
    T.AddToken(3, begin=False),
    T.ToTensor(),
    T.PadTransform(max_length=10, pad_value=0)
)
transform2 = T.Sequential(
    T.VocabTransform(en_vocab),
    T.Truncate(max_seq_len=8),
    T.AddToken(2, begin=True),
    T.AddToken(3, begin=False),
    T.ToTensor(),
    T.PadTransform(max_length=10, pad_value=0)
)
dataset_train = RuEnDataset(X_train, y_train, transform1, transform2)
dataset_train[0]

(tensor([   2,   27,  705, 2327, 5744,    3,    0,    0,    0,    0]),
 tensor([   2,    6,  150,  243,   24, 4026,    3,    0,    0,    0]))

In [ ]:
dataset_test = RuEnDataset(X_test, y_test, transform1, transform2)
dataset_test[0]

(tensor([  2,   5,  19,  53, 142,  84,   1,   3,   0,   0]),
 tensor([ 2, 10, 19, 20,  4, 39,  1, 79, 49,  3]))

## Создание модели `Encoder`, которая возвращает скрытое состояние рекуррентного слоя в соотстветствии со следующей схемой.

![encoder](https://i0.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-1.png?w=815&ssl=1)


In [ ]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size):
      super().__init__()
      self.emb = nn.Embedding(
          num_embeddings=vocab_size,
          embedding_dim=embedding_dim,
          padding_idx=0
      )
      self.dropout = nn.Dropout(p=0.5)
      self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)

  def forward(self, X):
    out = self.emb(X) # batch x seq x emb_size
    out = self.dropout(out)
    _, h = self.rnn(out) # out: batch x seq x hidden_size
    return h # 1 x batch x hidden_size

In [ ]:
embedding_dim = 100
encoder_hidden_size = 300
encoder = Encoder(len(ru_vocab), embedding_dim, encoder_hidden_size)
encoder_output = encoder(torch.stack(dataset_train[:16][0]))

In [ ]:
encoder_output.shape

torch.Size([1, 16, 300])

## Создание модели `Decoder`, которая возвращает прогноз (набор индексов слов на английском языке)

![decoder](https://i2.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-2.png?w=899&ssl=1)

In [ ]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, decoder_hidden_size):
    super().__init__()
    self.emb = nn.Embedding(
          num_embeddings=vocab_size,
          embedding_dim=embedding_dim,
          padding_idx=0
    )
    self.rnn = nn.GRUCell(embedding_dim, decoder_hidden_size)
    self.fc = nn.Linear(decoder_hidden_size, vocab_size)

  def forward(self, encoder_output, labels, teacher_forcing_ratio):
    # labels: batch x seq_len
    # encoder_output: 1 x batch x encoder_hidden_size
    seq_len = labels.size(1)
    input_tokens = labels[:, 0]
    decoder_hidden = encoder_output[0]
    predicts = []
    for _ in range(1, seq_len):
      out = self.emb(input_tokens).relu() # batch x emb_size
      decoder_hidden = self.rnn(out, decoder_hidden) # batch x dec_hidden
      out = self.fc(decoder_hidden) # batch x n_en_tokens
      # teacher forcing
      if random.random() < teacher_forcing_ratio:
          input_tokens = labels[:, _]
      else:
          input_tokens = out.argmax(dim=1).detach()
      predicts.append(out.unsqueeze(1))
    predicts = torch.cat(predicts, dim=1)
    return predicts # batch x seq x n_en_token

In [ ]:
decoder = Decoder(vocab_size=len(en_vocab), embedding_dim=100, decoder_hidden_size=300)
decoder_output = decoder(encoder_output, torch.stack(dataset_train[:16][1]))
decoder_output.shape

torch.Size([16, 9, 4264])

## Объединение моделей `Encoder` и `Decoder` в одну модель `EncoderDecoder`


In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, X, labels, teacher_forcing_ratio=0.5):
        encoder_output = self.encoder(X)
        decoder_output = self.decoder(encoder_output, labels, teacher_forcing_ratio)
        return decoder_output

In [ ]:
encoder = Encoder(vocab_size=len(ru_vocab), embedding_dim=100, hidden_size=300)
decoder = Decoder(vocab_size=len(en_vocab), embedding_dim=100, decoder_hidden_size=300)
encoder_decoder = EncoderDecoder(encoder, decoder)

In [ ]:
result = encoder_decoder(torch.stack(dataset_train[:16][0]), torch.stack(dataset_train[:16][1]))
result.shape

torch.Size([16, 9, 4264])

In [ ]:
result_new = result.view(-1, result.size(-1))
result_new.shape

torch.Size([144, 4264])

## Решение задачи классификации на основе прогнозов модели `EncoderDecoder`

Во время обучения выведем на экран значения функции потерь для эпохи (на обучающем множестве), значение accuracy по токенам (на обучающем множестве) и пример перевода, сгенерированного моделью.

In [ ]:
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=True)

In [ ]:
encoder = Encoder(vocab_size=len(ru_vocab), embedding_dim=100, hidden_size=300)
decoder = Decoder(vocab_size=len(en_vocab), embedding_dim=100, decoder_hidden_size=300)
encoder_decoder = EncoderDecoder(encoder, decoder)

criterion = nn.CrossEntropyLoss(ignore_index=0) # игнорирование индекса '<PAD>'
optimizer = optim.Adam(encoder_decoder.parameters(), lr=0.001)
epochs = 100

In [ ]:
%%time
for epoch in range(epochs):
    total_loss = 0
    total_tokens = 0
    correct_tokens = 0
    encoder_decoder.train()
    for input_tensor, labels in train_loader:
        optimizer.zero_grad()
        output = encoder_decoder(input_tensor, labels)
        loss = criterion(output.view(-1, output.size(-1)), labels[:, 1:].flatten())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        no_pad_tokens = (labels[:, 1:] != 0).sum().item()
        total_tokens += no_pad_tokens
        correct_tokens += ((output.argmax(dim=2) == labels[:, 1:]) * (labels[:, 1:] != 0)).sum().item()

    accuracy = correct_tokens / total_tokens * 100 # расчет точности перевода
    if ((epoch + 1) % 5) == 0:
        print(f"Epoch [{epoch + 1}/{epochs}] Loss: {(total_loss / len(train_loader)):.4f} Accuracy: {accuracy:.2f}%")

        encoder_decoder.eval()
        with torch.no_grad(): # вывод примера
            example_input = random.choice(dataset_train)[0]
            example_output = encoder_decoder(example_input.unsqueeze(0), example_input.unsqueeze(0), teacher_forcing_ratio=0)
            translation = example_output.argmax(dim=2).squeeze().tolist()
            print("Оригинал:", ' '.join(ru_vocab.lookup_tokens(example_input.tolist())))
            print("Перевод:", ' '.join(en_vocab.lookup_tokens(translation)))

Epoch [5/100] Loss: 3.4162 Accuracy: 41.92%
Оригинал: <SOS> какой город был переименован в 1968 г в <EOS>
Перевод: which city is the the of the the <EOS>
Epoch [10/100] Loss: 2.6606 Accuracy: 46.28%
Оригинал: <SOS> на чем играет игорь растеряев <EOS> <PAD> <PAD> <PAD>
Перевод: what does bashmet play play on <EOS> <EOS> <EOS>
Epoch [15/100] Loss: 2.1622 Accuracy: 50.42%
Оригинал: <SOS> сколько команд участвовало в чм по футболу 2018 <EOS>
Перевод: how many points participated in the chicago the <EOS>
Epoch [20/100] Loss: 1.7881 Accuracy: 54.62%
Оригинал: <SOS> какой автомобильной компании принадлежит марка плимут <EOS> <PAD> <PAD>
Перевод: what metal of the the plymouth <EOS> <EOS> <EOS>
Epoch [25/100] Loss: 1.4782 Accuracy: 59.37%
Оригинал: <SOS> в каком польском городе зародилось движение солидарность <EOS> <PAD>
Перевод: in which city city was the schwarzenegger of <EOS>
Epoch [30/100] Loss: 1.2265 Accuracy: 64.72%
Оригинал: <SOS> чем наводили румяна девушки в древней руси <EOS> <PAD

**Для расчета качества машинного перевода воспользуемся метрикой BLEU**

In [ ]:
originals = []
translations = []
encoder_decoder.eval()
with torch.no_grad():
    for input_tensor, labels in test_loader:
        output = encoder_decoder(input_tensor, input_tensor, teacher_forcing_ratio=0)
        translation = output.argmax(dim=2).tolist()
        labels_w, translation_w = [], []
        translations.extend(list(map(lambda x: en_vocab.lookup_tokens(x), translation)))
        originals.extend(list(map(lambda x: [en_vocab.lookup_tokens(x)], labels.tolist())))

bleu = bleu_score(translations, originals)
print("BLEU Score:", bleu)

BLEU Score: 0.20942311965760824
